In [1]:
#Neural network implementation
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from sklearn.model_selection import KFold
from keras.layers.core import Dense, Activation, Flatten, Dropout
from process_data_helper import (
    load_data,
    process_training_data,
    process_testing_data,
    write_predictions,
    process_data
)

/Users/jiexinchen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_data = load_data("training_data.txt")
test_data = load_data("test_data.txt")
X_train, X_test, Y_train = process_data(train_data, test_data, 'tfidf', add_bias=False)
X_train_l1norm, Y_train_l1norm = process_training_data(train_data, 'l1', add_bias = False)


In [18]:
#print(Y_train)
print(Y_train.sum())
print(Y_train_l1norm.sum())
print(np.amax(X_train))
print(np.amax(X_train_l1norm))
print(np.amax((X_train-X_train_l1norm)))
#X_train = X_train_l1norm
'''
thoughts for next-step:
1. try l1 norm(applied to training and test together) for NN
2. try different activation function
3. try different layer numbers

to-check:
1. processed data:
    are the X too small to be learned? -> try scale them up?
2. output prediction: are they always 0/1?

10107.0
10107.0
1.0
1.0
0.7246918984798962


In [55]:
L1 = 70
L2 = 25
L3 = 10
dropout_prob1 = 0.13
dropout_prob2 = 0.08
dropout_prob3 = 0.0
ACTIVATION = 'relu'
model = []
model = Sequential()
model.add(Dense(L1, input_shape=(X_train.shape[1],)))
model.add(Activation(ACTIVATION))
model.add(Dropout(dropout_prob1))

'''
model.add(Dense(L2))
model.add(Activation(ACTIVATION))
model.add(Dropout(dropout_prob2))
'''
'''
model.add(Dense(L3))
model.add(Activation(ACTIVATION))
model.add(Dropout(dropout_prob3))
'''
#the output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))
## Printing a summary of the layers and weights in your model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_208 (Dense)            (None, 70)                70070     
_________________________________________________________________
activation_208 (Activation)  (None, 70)                0         
_________________________________________________________________
dropout_113 (Dropout)        (None, 70)                0         
_________________________________________________________________
dense_209 (Dense)            (None, 1)                 71        
_________________________________________________________________
activation_209 (Activation)  (None, 1)                 0         
Total params: 70,141
Trainable params: 70,141
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
fit = model.fit(X_train, Y_train, batch_size=64, epochs=4,
    verbose=1)

Epoch 1/4
20000/20000 [==============================] - 3s 132us/step - loss: 0.4424 - acc: 0.7865
Epoch 2/4
20000/20000 [==============================] - 2s 97us/step - loss: 0.3230 - acc: 0.8605
Epoch 3/4
20000/20000 [==============================] - 2s 95us/step - loss: 0.2881 - acc: 0.8792
Epoch 4/4
20000/20000 [==============================] - 2s 94us/step - loss: 0.2464 - acc: 0.9039


In [57]:
#test_data = load_data("test_data.txt")
#X_test = process_testing_data(test_data, 'tfidf', add_bias=False)
write_predictions(model, X_train, "train_predictions.txt")
write_predictions(model, X_test, "test_predictions.txt")

In [71]:
#KFold method to estimate the test error
KFOLD = 5
kf = KFold(n_splits = KFOLD)
inds = [ind for ind in kf.split(X_train, Y_train)]
train_err = np.zeros((KFOLD, ))
test_err = np.zeros((KFOLD, ))
train_accuracy = np.zeros((KFOLD, ))
test_accuracy = np.zeros((KFOLD, ))
err_train_total = 0
err_val_total = 0
model_Kfold = []
fit = []
model_kfold = []
'''
L1 = 450
L2 = 150
L3 = 15
dropout_prob1 = 0.14
dropout_prob2 = 0.1
dropout_prob3 = 0.0
'''
for kfold_iter in range(KFOLD):
    model_kfold.append(Sequential())
    model_kfold[kfold_iter].add(Dense(L1, input_shape=(X_train.shape[1],)))
    model_kfold[kfold_iter].add(Activation('relu'))
    model_kfold[kfold_iter].add(Dropout(dropout_prob1))
    '''
    model_kfold[kfold_iter].add(Dense(L2))
    model_kfold[kfold_iter].add(Activation('relu'))
    model_kfold[kfold_iter].add(Dropout(dropout_prob2))
    '''
    '''
    model_kfold[kfold_iter].add(Dense(L3))
    model_kfold[kfold_iter].add(Activation('relu'))
    dropout_prob = dropout_prob3
    model_kfold[kfold_iter].add(Dropout(dropout_prob3))
    '''
    #the output layer
    model_kfold[kfold_iter].add(Dense(1))
    model_kfold[kfold_iter].add(Activation('sigmoid'))
    train, val = inds[kfold_iter]
    X_train_Kfold = X_train[train, :]
    Y_train_Kfold = Y_train[train, :]
    X_test_Kfold = X_train[val, :]
    Y_test_Kfold = Y_train[val, :]
    model_kfold[kfold_iter].compile(loss='binary_crossentropy',
                         optimizer='RMSprop', metrics=['accuracy'])
    fit.append(model_kfold[kfold_iter].fit(X_train_Kfold, Y_train_Kfold, batch_size=64, epochs=5,
                    verbose=1))
    train_score = model_kfold[kfold_iter].evaluate(X_train_Kfold, Y_train_Kfold, verbose=0)
    test_score = model_kfold[kfold_iter].evaluate(X_test_Kfold, Y_test_Kfold, verbose=0)
    train_err[kfold_iter] = train_score[0]
    test_err[kfold_iter] = test_score[0]
    train_accuracy[kfold_iter] = train_score[1]
    test_accuracy[kfold_iter] = test_score[1]
#err_train_total += train_err
#err_val_total += val_err
#err_train[dim_iter, N_iter] = err_train_total/5
#err_val[dim_iter, N_iter] = err_val_total/5

#score = modelE.evaluate(X_test_vec, y_test_onehot, verbose=0)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])


Epoch 1/5
16000/16000 [==============================] - 6s 348us/step - loss: 0.5155 - acc: 0.8124
Epoch 2/5
16000/16000 [==============================] - 1s 78us/step - loss: 0.3453 - acc: 0.8561
Epoch 3/5
16000/16000 [==============================] - 1s 76us/step - loss: 0.3180 - acc: 0.8672
Epoch 4/5
16000/16000 [==============================] - 1s 83us/step - loss: 0.3081 - acc: 0.8698
Epoch 5/5
16000/16000 [==============================] - 1s 86us/step - loss: 0.3051 - acc: 0.8714
Epoch 1/5
16000/16000 [==============================] - 6s 358us/step - loss: 0.5178 - acc: 0.8004
Epoch 2/5
16000/16000 [==============================] - 1s 79us/step - loss: 0.3415 - acc: 0.8594
Epoch 3/5
16000/16000 [==============================] - 1s 89us/step - loss: 0.3147 - acc: 0.8668
Epoch 4/5
16000/16000 [==============================] - 1s 86us/step - loss: 0.3071 - acc: 0.8679
Epoch 5/5
16000/16000 [==============================] - 1s 89us/step - loss: 0.3028 - acc: 0.8716
Epoch 1/

In [72]:
print(np.mean(train_accuracy))
print(np.mean(test_accuracy))
print(train_err)
print(test_err)

0.880175
0.8501000000000001
[0.28942222 0.28727808 0.28513491 0.28351232 0.2889    ]
[0.32588966 0.34272728 0.35206293 0.35486643 0.33979274]


In [51]:
print(train.shape)
print(val.shape)
print(X_train_Kfold.shape)
print(Y_train_Kfold.shape)
print(X_test_Kfold.shape)
print(Y_test_Kfold.shape)


(16000,)
(4000,)
(16000, 1000)
(16000, 1)
(4000, 1000)
(4000, 1)


In [77]:
inds

[(array([ 4000,  4001,  4002, ..., 19997, 19998, 19999]),
  array([   0,    1,    2, ..., 3997, 3998, 3999])),
 (array([    0,     1,     2, ..., 19997, 19998, 19999]),
  array([4000, 4001, 4002, ..., 7997, 7998, 7999])),
 (array([    0,     1,     2, ..., 19997, 19998, 19999]),
  array([ 8000,  8001,  8002, ..., 11997, 11998, 11999])),
 (array([    0,     1,     2, ..., 19997, 19998, 19999]),
  array([12000, 12001, 12002, ..., 15997, 15998, 15999])),
 (array([    0,     1,     2, ..., 15997, 15998, 15999]),
  array([16000, 16001, 16002, ..., 19997, 19998, 19999]))]